In [1]:
# IMPORTS
##########################

import agent
import environment
import doubledqn
import tools
import memory

import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import time

Using TensorFlow backend.


In [ ]:
import importlib
importlib.reload(agent)
importlib.reload(environment)
importlib.reload(doubledqn)
importlib.reload(tools)
importlib.reload(memory);

In [2]:
# MAIN
##################################

num_actions = 2
state_shape = (1,11) # State var in rows
memory_size = 20000
gamma = 0.9
target_update_frequency = 30000
num_init_samples_mem = 1000
batch_size = 50
max_episode_length = 100000
optimizer = 'adam'
loss = "mse"
eps = 0.1
env_name = "Simple_Cross"
experiment_id = "Test_linear_model"
monitoring = True # Store variables for TensorBoard monitoring and model_checkpoints

# Define logs directory if monitoring enabled
if monitoring:
    output_dir = tools.get_output_folder("./Logs",experiment_id)
    summary_writer = tf.summary.FileWriter(logdir=output_dir)
else:
    output_dir = None
    summary_writer = None

# Initialize Q-networks (value and target)
q_network = agent.get_model('simple',(state_shape[1],),num_actions)
target_q_network = agent.get_model('simple',(state_shape[1],),num_actions)

# Initialize environment
sumo_env =  environment.Env(    "cross.net.xml",
                                "cross.rou.xml",
                                state_shape,
                                num_actions,
                                use_gui=False
                           )

# Initialize replay memory
mem = memory.ReplayMemory(    memory_size,
                                 state_shape,
                                 num_actions
                             )

# Initialize Double DQN algorithm
ddqn = doubledqn.DoubleDQN(     q_network,
                                target_q_network,
                                mem,
                                gamma,
                                target_update_frequency,
                                num_init_samples_mem,
                                batch_size,
                                optimizer,
                                loss,
                                max_episode_length,
                                sumo_env,
                                output_dir,
                                experiment_id,
                                summary_writer
                            )

# Fill Replay Memory
ddqn.fill_replay(sumo_env)

Instructions for updating:
Colocations handled automatically by placer.
Filling experience replay memory...
...Done


In [3]:
# Train
_ = ddqn.train(  sumo_env, 300, "epsGreedy", eps=eps)

Running episode 1 / 1000WARNING:tensorflow:From /home/jrd/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
Running episode 827 / 1000

KeyboardInterrupt: 

In [ ]:
sumo_env =  environment.Env(    "cross.net.xml",
                                "cross.rou.xml",
                                state_shape,
                                num_actions,
                                use_gui=True
                           )
ddqn.train(  sumo_env, 1, "epsGreedy", eps=eps)
tools.compute_mean_duration(output_dir)

In [ ]:
#Testing model

dur_model = []
dur_static = []

for i in range(10):

    tools.generate_routefile()
    sumo_env =  environment.Env(    "cross.net.xml",
                                    "cross.rou.xml",
                                    state_shape,
                                    num_actions,
                                    use_gui=False
                               )
    #__,a=ddqn.evaluate(sumo_env,"greedy")
    __,b= ddqn.evaluate(sumo_env,"fixed", v_row_t = 40, h_row_t = 40)
    
    #dur_model.append(a)
    dur_model.append(b)

In [ ]:
plt.plot(range(10), np.hstack(dur_model,dur_static))

In [ ]:
dur_model

In [ ]:
import pandas as pd
pd.DataFrame(data)

In [ ]:
pd.DataFrame(data).state[22]